# 🤖 Modelado Predictivo del Dataset

En esta sección se construyen modelos predictivos para estimar la probabilidad de éxito (*target*) de cada candidato.  
El objetivo es:

- Entrenar modelos supervisados que capturen la relación entre variables explicativas y target
- Evaluar la importancia de cada variable
- Comparar distintas técnicas y seleccionar la mejor
- Preparar submodelos según área académica (Medicina, Enfermería, resto)

El proceso se divide en los siguientes pasos:
1. Preparación de datos y división en entrenamiento/test  
2. Selección de modelos factibles  
3. Selección de variables importantes  
4. Entrenamiento de submodelos  
5. Optimización de hiperparámetros  
6. Evaluación del modelo sobre conjunto test

---

## 1️⃣ Preparación de Datos

Antes de entrenar los modelos:

- Se eliminan variables irrelevantes o identificadores
- Se manejan valores nulos
- Se divide el dataset en **conjunto de entrenamiento** (70–80 %) y **conjunto de test** (20–30 %)
- Se crean subdatasets si se requiere segmentación por área académica

📌 Esto asegura que el modelo no vea datos de test durante el entrenamiento, evitando sobreajuste.

---

## 2️⃣ Selección de Modelos

Se consideran modelos supervisados robustos para clasificación/regresión según el target:

- **Random Forest**: Modelo basado en árboles con gran capacidad de generalización  
- **Gradient Boosting Machines (GBM/XGBoost/LightGBM)**: Optimización secuencial de errores  
- **Regresiones**: Lineales o logísticas, útiles para interpretación  
- Otros modelos factibles según el caso (SVM, redes neuronales simples)

Se busca evaluar cuál combina mejor:

- Precisión o % acierto
- AUC (Área bajo la curva ROC)
- Interpretabilidad
- Robustez ante outliers y variables correlacionadas

---

## 3️⃣ Selección de Variables Importantes

- Se utiliza la importancia que cada modelo asigna a las variables para:

  - Reducir dimensionalidad
  - Evitar ruido
  - Mejorar interpretabilidad

- Variables con mayor impacto en la predicción serán prioridad para la construcción de submodelos.

📌 Ejemplo: `feature_importances_` en Random Forest o `SHAP values` para interpretabilidad avanzada.

---

## 4️⃣ Entrenamiento de Submodelos

- Se pueden crear modelos específicos para segmentos de interés:  
  - Medicina  
  - Enfermería  
  - Resto de candidatos

- Esto permite capturar patrones particulares de cada área que un modelo general podría diluir.

---

## 5️⃣ Optimización de Hiperparámetros

- Se aplican técnicas de tuning automático (grid search, random search o frameworks como **PyCaret**)  
- Se buscan combinaciones que maximizan métricas clave:  

  - Precisión / Recall  
  - AUC / ROC  
  - F1-score  

- El objetivo es **obtener el mejor modelo posible** sin sobreajustar a los datos de entrenamiento.

---

## 6️⃣ Evaluación en Conjunto de Test

- Una vez seleccionado el modelo óptimo, se evalúa sobre datos no vistos:

  - Comprobando métricas globales: % acierto, AUC  
  - Detectando posibles sesgos por segmento  
  - Verificando estabilidad y consistencia de predicciones

- Esto garantiza que el modelo generalice y pueda usarse para predicciones futuras.

---

## 🔍 Conclusiones del Modelado

El modelado supervisado permite:

- Predecir con alta fiabilidad la probabilidad de éxito de los candidatos  
- Entender qué variables tienen mayor impacto en la conversión  
- Construir estrategias segmentadas por áreas académicas  
- Complementar el análisis de clusters previo, combinando insights no supervisados y supervisados

📌 Los resultados servirán como base para:

- Implementación de modelos en producción
- Definición de estrategias de admisión diferenciadas
- Análisis de riesgo y priorización de candidatos


In [19]:
# ============================================================
# SCRIPT · 03 MODELADO PREDICTIVO CON PYCARET (FINAL EXPLICABLE)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from pycaret.classification import (
    setup, compare_models, tune_model,
    predict_model, plot_model, save_model
)

# ============================================================
# 0️⃣ CARGA DEL DATASET
# ============================================================

ruta_dataset = r"..\datos\03. Datos analizados\dataset_clusterizado.csv"
df = pd.read_csv(ruta_dataset, sep=";")

target = 'target'

# ============================================================
# 0.1️⃣ LIMPIEZA BÁSICA
# ============================================================

# Eliminar IDs (no informativos)
cols_id = ['ACCOUNTID', 'ID', 'ID18__PC']
df = df.drop(columns=[c for c in cols_id if c in df.columns])

# Eliminar filas sin target
df = df.dropna(subset=[target])

# Sustituir NaN por 0
df = df.fillna(0)

# ============================================================
# 1️⃣ SELECCIÓN DE VARIABLES
# ============================================================

vars_excluir = [
    'cluster', 'interpretacion_cluster', 'PCA1', 'PCA2',
    'PORCENTAJE_PAGADO_FINAL', 'PAID_AMOUNT',
    'tiempo_etapa_dias', 'tiempo_entre_etapas_dias'
]

vars_modelado = [c for c in df.columns if c not in vars_excluir + [target]]

X = df[vars_modelado]
y = df[target]

print(f"Variables utilizadas ({len(vars_modelado)}):")
print(vars_modelado)

# ============================================================
# 2️⃣ TRAIN / TEST SPLIT
# ============================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ============================================================
# 3️⃣ FILTRADO DE COLUMNAS PROBLEMÁTICAS
# ============================================================

object_cols = X_train.select_dtypes(include=['object']).columns.tolist()
const_cols = X_train.columns[X_train.nunique() <= 1].tolist()

safe_cols = [c for c in X_train.columns if c not in object_cols + const_cols]

train_df = X_train[safe_cols].copy()
train_df[target] = y_train

# ============================================================
# 4️⃣ SETUP PYCARET (RÁPIDO + EXPLICABLE)
# ============================================================

setup(
    data=train_df,
    target=target,
    session_id=42,
    normalize=True,
    transformation=False,
    fold=3,               # rápido
    verbose=False,
    html=False
)

# ============================================================
# 5️⃣ COMPARACIÓN DE MODELOS EXPLICABLES
# ============================================================

best_models = compare_models(
    include=['lr', 'dt', 'rf', 'lightgbm'],
    sort='AUC',
    n_select=3,
    turbo=True
)

print("\nModelos seleccionados:")
for m in best_models:
    print(m)

# ============================================================
# 6️⃣ TUNING LIGERO
# ============================================================

tuned_models = [
    tune_model(m, optimize='AUC', n_iter=10, choose_better=True)
    for m in best_models
]

# ============================================================
# 7️⃣ VALIDACIÓN EN TEST + PROBABILIDADES
# ============================================================

test_df = X_test[safe_cols].copy()
test_df[target] = y_test

resultados = []

for model in tuned_models:
    preds = predict_model(model, data=test_df)

    auc = roc_auc_score(
        preds[target],
        preds['prediction_score']
    )

    prob_media = preds['prediction_score'].mean()

    resultados.append({
        'modelo': str(model),
        'AUC_test': round(auc, 3),
        'prob_media_exito': round(prob_media, 3)
    })

    print(f"\nModelo: {model}")
    print(f"AUC test: {auc:.3f}")
    print(f"Probabilidad media de éxito: {prob_media:.3f}")

# ============================================================
# 8️⃣ IMPORTANCIA DE VARIABLES (MODELO FINAL)
# ============================================================

best_model = tuned_models[0]

# Importancia nativa
plot_model(best_model, plot='feature')

# Importancia robusta (permutation)
plot_model(best_model, plot='feature_all')

# ============================================================
# 9️⃣ RESUMEN FINAL DE MODELOS
# ============================================================

resumen_df = pd.DataFrame(resultados)
print("\nResumen comparativo de modelos:")
print(resumen_df)

# ============================================================
# 🔟 GUARDAR MODELO FINAL
# ============================================================

save_model(best_model, r"..\datos\04. Modelos\modelo_final_explicable")

print("\nModelo final guardado correctamente.")


FileNotFoundError: [Errno 2] No such file or directory: '..\\datos\\03. Datos analizados\\dataset_clusterizado.csv'